# Albeno effekt
---
På bakgrunn av hva vi lærte i oppgave 2 rundt datasettet har vi funnet ut:      
- Alpene er bra for å se på snøsmelting.
- At 21 juni er bra for å se på snøsmelting fra år til år, siden solen er på sitt sterkeste da.
- At AL-BB-DH passer vår oppgave best ved at vi ser på et tidspunkt med lite skydekke.
- At AL-BB-DH-ERR gir også best data over de 21 årene, ved å sammenligne år til år. 

Kode 1 filtrer informasjonen ovenfor inn i en csv og komprimerer fil plasseringen til de orginale netcdf filene på 400MB til csv filer på 17MB

#### Kode 1


### Kode 1

Denne koden koprimer dataen vi trenger til csv slikt at vi kan bruke sql for å undersøke datasettet og videre bruk.

Lagrer dataen i "data" med navnet csv_albedo_effekt

In [18]:
import os
import xarray as xr
import pandas as pd

# Lat og lon for alpene
y_min, y_max = 640, 715
x_min, x_max = 1700, 1900

# Fil plassering csv
csv_folder = os.path.join("..", "data", "csv_albedo_effekt")
os.makedirs(csv_folder, exist_ok=True)

# For løkke fra årene fra 2004 til 2024
for year in range(2004, 2025):
    # Filnavn og filsti.
    file_name = f"NETCDF4_LSASAF_MSG_ALBEDO-D10v2_MSG-Disk_{year}06250000.nc"
    file_path = os.path.join("..", "data", "albedo_effekt_data", file_name)
    
    # Åpne NetCDF-filen
    ds = xr.open_dataset(file_path)
    
    # Utsnitt av datasettet for å fokusere på regionen av interesse
    ds_utsnitt = ds.isel(lat=slice(y_min, y_max), lon=slice(x_min, x_max))
    
    # Ønskede variabler: AL-BB-DH, AL-BB-DH-ERR og quality_flag
    variables = ["AL-BB-DH", "AL-BB-DH-ERR", "quality_flag"]
    
    # Konverter de valgte variablene til en DataFrame
    df = ds_utsnitt[variables].to_dataframe().reset_index()
    
    # Sett sammen CSV-filnavnet med årstall
    csv_file_name = f"Albedo effekt {year}.csv"
    csv_file_path = os.path.join(csv_folder, csv_file_name)
    
    # Lagre DataFrame til CSV
    df.to_csv(csv_file_path, index=False)
    print(f"Data for {year} lagret som {csv_file_path}")

Data for 2004 lagret som ..\data\csv_albedo_effekt\Albedo effekt 2004.csv
Data for 2005 lagret som ..\data\csv_albedo_effekt\Albedo effekt 2005.csv
Data for 2006 lagret som ..\data\csv_albedo_effekt\Albedo effekt 2006.csv
Data for 2007 lagret som ..\data\csv_albedo_effekt\Albedo effekt 2007.csv
Data for 2008 lagret som ..\data\csv_albedo_effekt\Albedo effekt 2008.csv
Data for 2009 lagret som ..\data\csv_albedo_effekt\Albedo effekt 2009.csv
Data for 2010 lagret som ..\data\csv_albedo_effekt\Albedo effekt 2010.csv
Data for 2011 lagret som ..\data\csv_albedo_effekt\Albedo effekt 2011.csv
Data for 2012 lagret som ..\data\csv_albedo_effekt\Albedo effekt 2012.csv
Data for 2013 lagret som ..\data\csv_albedo_effekt\Albedo effekt 2013.csv
Data for 2014 lagret som ..\data\csv_albedo_effekt\Albedo effekt 2014.csv
Data for 2015 lagret som ..\data\csv_albedo_effekt\Albedo effekt 2015.csv
Data for 2016 lagret som ..\data\csv_albedo_effekt\Albedo effekt 2016.csv
Data for 2017 lagret som ..\data\csv_a

### Kode 2
I koden undersøker vi data kordinater som gir nøyaktig informasjon i alle de 21 årene.  
Vi bruker : 
Query, fra pandassql  
List comprehension for years og coords   
Og en Iterator i filtered_coords

Totalt antall områder med feil < 0.01 (1% feil) i alle år: 11573 kordinater.    
Koordinatene med best data lagret i ..\data\csv_albedo_effekt\data_m-lavfeilmargin.csv

### Kommentar
Ved visualiseringen i mappe 2 kan vi nå velge kordinater fra denne nye csv filen, med høy nøyaktighet


In [19]:
import os
import pandas as pd
import pandasql as psql

# Fil path til CSV-filene
csv_folder = os.path.join("..", "data", "csv_albedo_effekt")

# Liste over årstall
years = [year for year in range(2004, 2025)]

# Variabel for å lagre de felles kvalifiserte koordinatene (lat, lon)
qualified_coords = None

# Funksjon som gir en iterator over filtrerte koordinater fra en fil
def filtered_coords(file_path):
    df = pd.read_csv(file_path)

    # SQL-spørring for å hente ut lat og lon med feil < 0.01
    query = """
    SELECT lat, lon
    FROM df
    WHERE `AL-BB-DH-ERR` < 0.01
    """
    result = psql.sqldf(query, locals())

    # Bruk en generator for å levere koordinater en etter en 
    for lat, lon in zip(result["lat"], result["lon"]):
        yield lat, lon

# Samle alle kvalifiserte koordinater fra hvert år
for year in years:
    file_path = os.path.join(csv_folder, f"Albedo effekt {year}.csv")

    # Bygger settet med koordinater
    coords = {coord for coord in fetch_filtered_coords(file_path)}
    print(f"{year}: {len(coords)} områder med feil < 0.01")
    
    # Finner felles koordinater mellom år
    qualified_coords = coords if qualified_coords is None else qualified_coords.intersection(coords)

print(f"\nTotalt antall områder med feil < 0.01 i alle år: {len(qualified_coords)}")

# Lagre resultatet i en CSV-fil i samme folder
output_file = os.path.join(csv_folder, "data_m-lavfeilmargin.csv")
df_result = pd.DataFrame(list(qualified_coords), columns=["lat", "lon"])
df_result.to_csv(output_file, index=False)
print(f"Koordinatene med best data lagret i {output_file}")

2004: 12691 områder med feil < 0.01
2005: 13254 områder med feil < 0.01
2006: 13553 områder med feil < 0.01
2007: 13099 områder med feil < 0.01
2008: 13365 områder med feil < 0.01
2009: 12861 områder med feil < 0.01
2010: 13376 områder med feil < 0.01
2011: 13511 områder med feil < 0.01
2012: 13219 områder med feil < 0.01
2013: 12134 områder med feil < 0.01
2014: 12806 områder med feil < 0.01
2015: 13434 områder med feil < 0.01
2016: 13150 områder med feil < 0.01
2017: 13455 områder med feil < 0.01
2018: 13024 områder med feil < 0.01
2019: 13281 områder med feil < 0.01
2020: 13410 områder med feil < 0.01
2021: 12911 områder med feil < 0.01
2022: 13637 områder med feil < 0.01
2023: 13322 områder med feil < 0.01
2024: 12566 områder med feil < 0.01

Totalt antall områder med feil < 0.01 i alle år: 11573
Koordinatene med best data lagret i ..\data\csv_albedo_effekt\data_m-lavfeilmargin.csv


---
# Oppgave 3 konklusjon

Vi har filtrert dataen ved hjelp av list comprehensions, iteratorer og Pandas SQL. 